# Asset Model Loader

A notebook to demonstrate an interface with some of the features of the Asset Model Loader class

Authors: gavin.treseder@essentialenergy.com.au; illyse.schram@essentialenergy.com.au

In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import copy
import pof.demo as demo
import pprint
pp = pprint.PrettyPrinter(indent=4)

from pof.loader.asset_model_loader import AssetModelLoader

In [2]:
filename = r"C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\pof\data\inputs\Asset Model - Demo.xlsx"


In [3]:
aml = AssetModelLoader()
data = aml.load(filename)

In [4]:
from pof.failure_mode import FailureMode
FailureMode().load(data['pole']['fm']['termites'])

Error loading FailureMode data


In [5]:
pp.pprint(data['pole']['fm']['termites'])

{   'conditions': {'external_diameter': None, 'wall_thickness': None},
    'name': 'termites',
    'tasks': {   'inspection': {   'activity': 'Inspection',
                                   'admin': 'yes',
                                   'cost': 50.0,
                                   'impacts': {   'condition': {},
                                                  'state': (   {   'detection': 1.0},)},
                                   'level_of_repair': 'as_bad_as_old',
                                   'maint': 'no',
                                   'name': 'inspection',
                                   'p_effective': 0.8,
                                   't_delay': 20.0,
                                   't_interval': 5.0,
                                   'task': 'inspection',
                                   'task_group_name': 'level_3_inspection',
                                   'travel': 'yes',
                                   'trigger': 'time',
          

In [6]:
df = pd.read_excel(filename, sheet_name='Model Input', header=[0,1,2])
df.head()

Unnamed: 0_level_0 asset_model                              \
  Unnamed: 0_level_1      system sub_system component          
                  id        name       name      name name.1   
0                NaN         NaN        NaN      pole    NaN   
1                NaN         NaN        NaN       NaN    NaN   
2                NaN         NaN        NaN       NaN    NaN   
3                NaN         NaN        NaN       NaN    NaN   
4                NaN         NaN        NaN       NaN    NaN   

        indicator_model                                    ... trigger_model  \
              indicator                                    ...     condition   
                   name      dist perfect failed failed.1  ...         upper   
0        wall_thickness    linear   asset    0.0      NaN  ...           NaN   
1     external_diameter    linear   asset    0.0      NaN  ...           NaN   
2  simple_safety_factor        sf       4    1.0      NaN  ...           max   
3  actual_safety_factor  dsf_calc   asset    1.0      NaN  ...           max   
4                   NaN       NaN     NaN    NaN      NaN  ...           NaN   

          impact_model                                               \
                 state                             condition          
  upper.1   initiation detection failure failure.1      name target   
0     NaN          NaN       1.0     NaN       NaN       NaN    NaN   
1     NaN          NaN       NaN     NaN       NaN       NaN    NaN   
2     NaN          0.0       0.0     0.0       NaN       all    0.0   
3     NaN          NaN       NaN     NaN       NaN       NaN    NaN   
4     NaN          NaN       NaN     NaN       NaN       NaN    NaN   

                                
                                
             method       axis  
0               NaN        NaN  
1               NaN        NaN  
2  reduction_factor  condition  
3               NaN        NaN  
4               NaN        NaN  

[5 rows x 53 columns]

In [7]:
df = df.dropna(axis=1, how='all')
df.head()

asset_model       indicator_model                          failure_model  \
    component             indicator                           failure_mode   
         name                  name      dist perfect failed          name   
0        pole        wall_thickness    linear   asset    0.0      termites   
1         NaN     external_diameter    linear   asset    0.0           NaN   
2         NaN  simple_safety_factor        sf       4    1.0           NaN   
3         NaN  actual_safety_factor  dsf_calc   asset    1.0           NaN   
4         NaN                   NaN       NaN     NaN    NaN           NaN   

                                    ...      trigger_model              \
  distribution                      ...          condition               
         alpha beta gamma pf_curve  ...               name lower upper   
0         50.0  1.5  10.0   linear  ...                NaN   NaN   NaN   
1          NaN  NaN   NaN      NaN  ...                NaN   NaN   NaN   
2          NaN  NaN   NaN      NaN  ...     wall_thickness  50.0   max   
3          NaN  NaN   NaN      NaN  ...  external_diameter  50.0   max   
4          NaN  NaN   NaN      NaN  ...                NaN   NaN   NaN   

  impact_model                                                                  
         state                   condition                                      
    initiation detection failure      name target            method       axis  
0          NaN       1.0     NaN       NaN    NaN               NaN        NaN  
1          NaN       NaN     NaN       NaN    NaN               NaN        NaN  
2          0.0       0.0     0.0       all    0.0  reduction_factor  condition  
3          NaN       NaN     NaN       NaN    NaN               NaN        NaN  
4          NaN       NaN     NaN       NaN    NaN               NaN        NaN  

[5 rows x 41 columns]

In [8]:
# Create keys
keys = [
    ('asset_model', 'component', 'name'),
    #('indicator_model', 'indicator','name'),
    ('failure_model', 'failure_mode', 'name'),
    ('task_model', 'task', 'name'),
    #('trigger_model', 'condition', 'name'), #TODO revist this one
    #('impact_model', 'condition', 'name'),
]

In [9]:
def validate_keys(keys, df):
    missing_keys = [key for key in keys if key not in df.columns]

    if bool(missing_keys):
        print("Missing Keys: %s" %(missing_keys))
        return False
    else:
        return True

validate_keys(keys, df)

True

In [10]:
# Propogate keys for 1 to many relationships
df[keys] = df[keys].ffill()

In [11]:

pp.pprint(demo.failure_mode_data['slow_aging'])

{   'conditions': {   'fast_degrading': {   'failed': 0,
                                            'name': 'fast_degrading',
                                            'perfect': 100,
                                            'pf_curve': 'linear',
                                            'pf_interval': 10,
                                            'pf_std': 0.5},
                      'slow_degrading': {   'failed': 0,
                                            'name': 'slow_degrading',
                                            'perfect': 100,
                                            'pf_curve': 'linear',
                                            'pf_interval': 20,
                                            'pf_std': 0.5}},
    'name': 'slow_aging',
    'states': {'detection': False, 'failure': False, 'initiation': False},
    'tasks': {   'inspection': {   'activity': 'Inspection',
                                   'cost': 50,
                                   'im

In [12]:
def get_failure_mode_data(df_comp):

    fms_data = dict() #TODO
    fm_key = ('failure_model', 'failure_mode', 'name')
    failure_modes = df_comp[fm_key].unique()
    df_fms = df_comp[['failure_model', 'condition_model', 'task_model', 'trigger_model', 'impact_model']].set_index(fm_key)

    for fm in failure_modes:

        df_fm = df_fms.loc[[fm]]

        # Get the Task information
        tasks_data = get_task_data(df_fm)
        
        # Get the Distribution information
        dist_data = get_dist_data(df_fm)

        # Get the Condition information
        condition_data = get_condition_data(df_fm)

        fm_data = dict(
            name = fm,
            conditions = condition_data,
            tasks = tasks_data,
            untreated = dist_data,
        )
        fms_data.update({
            fm : fm_data,
        })

    return fms_data

In [13]:
def get_condition_data(df_fm):
    #TODO update for new arrangement
    df_cond = df_fm['condition_model']
    conditions = df_cond['condition']['name'].dropna().to_numpy()
    cond_data = {cond : None for cond in conditions}
    return cond_data

In [19]:
def get_dist_data(df_fm):

    df_dist = df_fm['failure_model'].dropna()
    df_dist.columns = df_dist.columns.droplevel()

    try:

        dist_data = df_dist.iloc[0].dropna().to_dict()
    except IndexError:
        dist_data = df_dist.dropna().to_dict()

    return dist_data

In [15]:
def get_task_data(df_fm):
    """Takes a dataframe for a failure mode and returns a dict of task data
    """
    tasks_data = dict()
    task_key = ('task_model', 'task', 'name')
    tasks = df_fm[task_key].unique()
    df_tasks = df_fm[['task_model', 'trigger_model', 'impact_model']].set_index(task_key)

    for task in tasks:

        df_task = df_tasks.loc[[task]].dropna(axis=0, how='all')

        # Trigger information
        try:
            state = df_task['trigger_model']['state'].iloc[0].dropna().to_dict(),
        except:
            state = df_task['trigger_model']['state'].dropna().to_dict(),

        trigger_data = dict(
            state = state,
            condition = df_task['trigger_model']['condition'].dropna().set_index('name').to_dict('index')
        )

        # Impact information
        try:
            state = df_task['impact_model']['state'].iloc[0].dropna().to_dict(),
        except:
            state = df_task['impact_model']['state'].dropna().to_dict(),
            
        impact_data = dict(
            state = state,
            condition = df_task['impact_model']['condition'].dropna().set_index('name').to_dict('index')
        )

        # Tasks specific information
        df_tsi = df_task[('task_model')].dropna(how='all').dropna(axis=1)
        df_tsi.columns = df_tsi.columns.droplevel()

        task_data = df_tsi.to_dict('index') #TODO currently has too many vars
        try:
            task_data[task].update(dict(
                name=task,
                triggers=trigger_data,
                impacts = impact_data,
            ))
        except:
            task_data = dict(
                task = dict(
                    name=task,
                    triggers=trigger_data,
                    impacts = impact_data,
                )
            )

        tasks_data.update(task_data)

    return tasks_data

In [26]:
def _get_component_data(df):
    comps_data = dict()

    # Get the Component information
    comp_key = ('asset_model', 'component', 'name')
    components = df[comp_key].dropna().unique()

    df_comps = df[['asset_model', 'failure_model', 'condition_model', 'task_model', 'trigger_model', 'impact_model']].set_index(comp_key)

    for comp in components:

        df_comp = df_comps.loc[[comp]]

        # Get the FailureMode information
        fm_data = get_failure_mode_data(df_comp)

        comp_data = dict(
            fm = fm_data,
        )

        comps_data.update({
            comp : comp_data
        })

    return comps_data

In [28]:
comp2 = _get_component_data(df)

{'alpha': 50.0, 'beta': 1.5, 'gamma': 10.0, 'pf_curve': 'linear', 'pf_interval': 20.0, 'pf_std': 1.0}
{'alpha': {}, 'beta': {}, 'gamma': {}, 'pf_curve': {}, 'pf_interval': {}, 'pf_std': {}}
{'alpha': {}, 'beta': {}, 'gamma': {}, 'pf_curve': {}, 'pf_interval': {}, 'pf_std': {}}


In [17]:
comps_data = dict()

# Get the Component information
comp_key = ('asset_model', 'component', 'name')
components = df[comp_key].unique()

df_comps = df[['asset_model', 'failure_model', 'condition_model', 'task_model', 'trigger_model', 'impact_model']].set_index(comp_key)

for comp in ['pole']:

    df_comp = df_comps.loc[[comp]]

    fms_data = dict() #TODO
    fm_key = ('failure_model', 'failure_mode', 'name')
    failure_modes = df_comp[fm_key].unique()
    df_fms = df_comp[['failure_model', 'condition_model', 'task_model', 'trigger_model', 'impact_model']].set_index(fm_key)

    for fm in ['termites']:
        
        df_fm = df_fms.loc[[fm]]
        # Get the Task information
        tasks_data = get_task_data(df_fm)
        
        # Get the Distribution information
        df_dist = df_fm['failure_model'].dropna()
        df_dist.columns = df_dist.columns.droplevel()

        try:

            dist_data = df_dist.iloc[0].dropna().to_dict()
        except IndexError:
            dist_data = df_dist.dropna().to_dict()




termites


In [18]:
dist_data

{'alpha': 50.0,
 'beta': 1.5,
 'gamma': 10.0,
 'pf_curve': 'linear',
 'pf_interval': 20.0,
 'pf_std': 1.0}

In [38]:
from pof.component import Component
from pof.failure_mode import FailureMode
from pof.task import Inspection, Task

FailureMode().load(demo.failure_mode_data['slow_aging'])
FailureMode().load(comps_data['pole']['fm']['termites'])

Inspection().load(demo.inspection_data['instant'])
Inspection().load(comps_data['pole']['fm']['termites']['tasks']['inspection'])

pp.pprint(demo.failure_mode_data['slow_aging'])

Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
Error loading FailureMode data
Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
Invalid Consequence - currently stubbed
{   'conditions': {   'fast_degrading': {   'failed': 0,
                                            'name': 'fast_degrading',
                                            'perfect': 100,
                                            'pf_curve': 'linear',
                                            'pf_interval': 10,
                                            'pf_std': 0.5},
                      'slow_degrading': {   'failed': 0,
                                            'name': 'slow_degrading',
                                            'perfect': 100,
      

In [37]:
pp.pprint(comps_data['pole']['fm']['termites'])

{   'conditions': {'external_diameter': None, 'wall_thickness': None},
    'name': 'termites',
    'tasks': {   'inspection': {   'activity': 'Inspection',
                                   'admin': 'yes',
                                   'cost': 50.0,
                                   'impacts': {   'condition': {},
                                                  'state': ({},),
                                                  'time': {}},
                                   'level_of_repair': 'as_bad_as_old',
                                   'maint': 'no',
                                   'name': 'inspection',
                                   'p_effective': 0.8,
                                   't_delay': 20.0,
                                   't_interval': 5.0,
                                   'task': 'inspection',
                                   'task_group_name': 'level_3_inspection',
                                   'travel': 'yes',
                        

In [342]:
pp.pprint(tasks_data)

{   'inspection': {   'Level of Failure': 'nil',
                      'admin': 'yes',
                      'cost': 50.0,
                      'impacts': {   'condition': {},
                                     'state': {   'detection': nan,
                                                  'failure': nan,
                                                  'initiation': nan}},
                      'level_of_repair': 'as_bad_as_old',
                      'maint': 'no',
                      'name': 'inspection',
                      'p_effective': 0.8,
                      't_delay': 20.0,
                      't_interval': 5.0,
                      'task': 'inspection',
                      'task_group_name': 'level_3_inspection',
                      'travel': 'yes',
                      'trigger': 'time',
                      'triggers': {   'condition': {},
                                      'state': {   'detection': 'any',
                                            

In [48]:

pp.pprint(demo.inspection_data['degrading'])

{   'activity': 'Inspection',
    'cost': 50,
    'impacts': {'condition': {}, 'state': {'detection': True}, 'time': {}},
    'name': 'inspection',
    'p_effective': 0.9,
    't_delay': 10,
    't_interval': 5,
    'triggers': {   'condition': {   'fast_degrading': {   'lower': 0,
                                                           'upper': 90},
                                     'slow_degrading': {   'lower': 0,
                                                           'upper': 90}},
                    'state': {'initiation': True},
                    'time': {}}}
